In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pdfplumber
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import fitz
from requests.exceptions import HTTPError
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
#set our pytesseract path

from PIL import Image

import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [13]:
import requests
import fitz
from requests.exceptions import HTTPError

def pdf_scraper(url):
    if url == "No articles found":
        return "No articles found"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/118.0.0.0 Safari/537.36"
        )
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except HTTPError as e:
        return f"Access denied ({e})"
    except Exception as e:
        return f"Request failed ({e})"

    pdf = response.content
    doc = fitz.open("pdf", pdf)
    text_parts = []

    if len(doc) > 20:
        text_parts.append("Too_Long")

    for k in range(min(len(doc), 20)):
        text_parts.append(doc[k].get_text())

    text = "".join(text_parts)
    if len(text)<200:
        #for z in range(len(doc)):
        for z in range(min(len(doc), 10)):
            page=doc[z]
            pix=page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            text= pytesseract.image_to_string(img)
            text_parts.append(text)
        doc.close()
        text="".join(text_parts)
        return text_parts
    if len(text) == 0:
        return "No text found"
    return text

In [ ]:

main_url="https://acrbgov.org/complaints/2023-complaints-reviewed/"
def url_scraper(main_url,year):
    headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/118.0.0.0 Safari/537.36"
            )
        }

    response=requests.get(main_url, headers=headers)
    r=response
    soup = BeautifulSoup(r.text, "html.parser")

    df = pd.DataFrame(columns=["Date", "P_tags", "LI_tags","URL_Text", "URLs", "Text"])

    # iterate only the larger sections you care about
    sections = soup.find_all("section")
    for section in sections:
        if len(section.find_all("p")) <= 10:
            continue

        # find inner blocks that contain the content groups
        blocks = section.find_all("section", class_="elementor-section")
        current_date=None
        for block in blocks:
            # state for the current group
            #current_date = None
            p_collected = []
            li_collected = []
            url_collected = []
            url_text=[]
            text=None
            # iterate elements in document order but only pay attention to p, li, a
            for el in block.find_all(['p', 'li', 'a'], recursive=True):
                # normalize text
                text = el.get_text(strip=True)

                # If this is a <p> that contains a date token (simple check for "2023")
                if el.name == "p" and str(year) in text:
                    # flush previous group if it had anything
                    if current_date or p_collected or li_collected or url_collected or url_text:
                        for i,url in enumerate(url_collected):
                            text=pdf_scraper(text)
                            df.loc[len(df)] = [current_date, p_collected, li_collected,url_text[i], url, text]
                            #print(url_text[i])
                        # reset collectors
                        p_collected = []
                        li_collected = []
                        url_collected = []
                        url_text=[]

                    # set new date and also store this <p> under p_collected
                    current_date = text
                    p_collected.append(text)
                    continue

                # If it's a plain <p> (no date) and we have a current_date, attach it
                if el.name == "p":
                    # store p text; also optionally store the associated date alongside it
                    # here we store as a tuple (date, p_text) so the date is explicit
                    p_collected.append(text)
                    continue

                # collect li items, tag them with current_date
                if el.name == "li":
                    li_collected.append(text)
                    continue

                # collect links
                if el.name == "a":
                    href = el.get("href")
                    if href:
                        url_collected.append(href)
                        url_text.append(text)
                        #print(text)
                    
                    continue

            # flush any remaining group at the end of the block
            if current_date or p_collected or li_collected or url_collected:
                if url_collected:
                    for i,url in enumerate(url_collected):
                        text=pdf_scraper(url)
                        df.loc[len(df)] = [current_date, p_collected, li_collected,url_text[i], url, text]
                    #print("2nd")
                    #print(url_text)    
                else:
                    df.loc[len(df)] = [current_date, p_collected, li_collected,url_text, url_collected, "No text here"]
    return df

In [ ]:
years=[ 2016]
def all_years_scraper(years):
    data=[]
    for year in years:
        new_url=f"https://acrbgov.org/complaints/{year}-complaints-reviewed/"
        z=url_scraper(new_url, year)
        #data.append(z)
        z.to_csv("small_table"+str(year)+".csv", chunksize=10000)
        print(year)
    combined=pd.concat(data, ignore_index=True)
    return combined
    #return("Success")

In [ ]:
a=all_years_scraper(years)
a.to_csv("ACRB_Data")

MemoryError: 